In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file_path = '/content/drive/MyDrive/Portfolio 6/3238357.zip'

In [ ]:
!unzip -q "$zip_file_path" -d "/content/drive/MyDrive/Portfolio 6"

replace /content/drive/MyDrive/Portfolio 6/images.tar.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
tar_file_path = '/content/drive/MyDrive/Portfolio 6/bounding_boxes.tar.gz'


In [ ]:
!tar -xvzf "$tar_file_path" -C "/content/drive/MyDrive/Portfolio 6"

In [ ]:
tar_file_path = '/content/drive/MyDrive/Portfolio 6/images.tar.gz'

In [ ]:
!tar -xvzf "$tar_file_path" -C "/content/drive/MyDrive/Portfolio 6"

images/train/IMG_20180723_123611.jpg
images/train/IMG_20180811_084218.jpg
images/train/IMG_0124.JPG
images/train/IMG_20180723_124259.jpg
images/train/IMG_20180725_115234.jpg
images/train/IMG_20180724_124548.jpg
images/train/IMG_20180719_124416.jpg
images/train/IMG_20180724_121704.jpg
images/test/IMG_1246.jpg
images/test/IMG_20180725_142219.jpg
images/train/IMG_20100101_030428.jpg
images/train/IMG_20180725_122605.jpg
images/train/IMG_20180720_124223.jpg
images/train/IMG_20180721_091154.jpg
images/train/IMG_20180724_120903.jpg
images/train/IMG_20180714_104446.jpg
images/train/IMG_20180714_102503.jpg
images/test/IMG_1263.jpg
images/test/IMG_20100101_033559.jpg
images/train/IMG_20180723_145937.jpg
images/train/IMG_20100101_032407.jpg
images/test/IMG_20100101_033158.jpg
images/train/IMG_1242.jpg
images/train/IMG_20180723_152110.jpg
images/train/IMG_20180811_084135.jpg
images/test/IMG_0234.JPG
images/train/IMG_20180717_120936.jpg
images/train/IMG_20100101_033626.jpg
images/train/IMG_20180721

In [ ]:
# Define directories
train_annotation_csv = '/content/drive/MyDrive/Portfolio 6/Bounding_boxes/train_labels.csv'
test_annotation_csv = '/content/drive/MyDrive/Portfolio 6/Bounding_boxes/test_labels.csv'
train_image_dir = '/content/drive/MyDrive/Portfolio 6/images/train'
test_image_dir = '/content/drive/MyDrive/Portfolio 6/images/test'
train_label_dir = '/content/drive/MyDrive/Portfolio 6/labels/train'
test_label_dir = '/content/drive/MyDrive/Portfolio 6/labels/test'
# Target directories for YOLO formatted data
train_target_image_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/train/images'
train_target_label_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/train/labels'
test_target_image_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/test/images'
test_target_label_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/test/labels'

In [ ]:
import os
import shutil
import pandas as pd

def convert_to_yolo_format(image_width, image_height, x_min, y_min, x_max, y_max):
    center_x = (x_min + x_max) / 2 / image_width
    center_y = (y_min + y_max) / 2 / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    return center_x, center_y, width, height

def generate_yolo_labels(annotation_csv, output_dir, image_dir):
    os.makedirs(output_dir, exist_ok=True)
    annotation_df = pd.read_csv(annotation_csv)

    for _, row in annotation_df.iterrows():
        image_name = row['filename']
        image_path = os.path.join(image_dir, image_name)

        if not os.path.exists(image_path):
            print(f"Warning: Image {image_name} not found. Skipping this annotation.")
            continue

        image_width, image_height = row['width'], row['height']
        x_min, y_min, x_max, y_max = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        # Convert to YOLO format
        center_x, center_y, width, height = convert_to_yolo_format(image_width, image_height, x_min, y_min, x_max, y_max)

        # Write the YOLO label to a .txt file
        label_filename = os.path.splitext(image_name)[0] + '.txt'
        label_filepath = os.path.join(output_dir, label_filename)

        with open(label_filepath, 'w') as label_file:
            label_file.write(f"{row['class']} {center_x} {center_y} {width} {height}\n")

generate_yolo_labels(train_annotation_csv, train_label_dir, train_image_dir)

generate_yolo_labels(test_annotation_csv, test_label_dir, test_image_dir)


In [ ]:
def count_label_files(label_dir, image_dir):
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    num_labels = len(label_files)
    num_images = len(image_files)

    print(f"Number of label files: {num_labels}")
    print(f"Number of image files: {num_images}")

    # Checking for any missing label files
    if num_labels < num_images:
        print("Warning: Some images do not have corresponding label files.")
    elif num_labels > num_images:
        print("Warning: More label files than images.")
    else:
        print("All images have corresponding label files.")

# Check labels for train and test sets
count_label_files(train_label_dir, train_image_dir)
count_label_files(test_label_dir, test_image_dir)


Number of label files: 813
Number of image files: 767
Number of label files: 209
Number of image files: 198


In [ ]:
import os
import shutil
import random

# Function to copy selected images and their corresponding YOLO labels to the target directory
def prepare_yolo_data(image_dir, label_dir, target_image_dir, target_label_dir, selected_images):
    os.makedirs(target_image_dir, exist_ok=True)
    os.makedirs(target_label_dir, exist_ok=True)

    for image in selected_images:
        image_path = os.path.join(image_dir, image)
        label_name = os.path.splitext(image)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        target_image_path = os.path.join(target_image_dir, image)
        target_label_path = os.path.join(target_label_dir, label_name)

        # Copy the image file to the target directory
        if os.path.exists(image_path):
            shutil.copy(image_path, target_image_path)
        else:
            print(f"Warning: Image file {image_path} not found, skipping.")

        # Copy the corresponding label file to the target directory
        if os.path.exists(label_path):
            shutil.copy(label_path, target_label_path)
        else:
            print(f"Warning: Label file {label_path} not found, skipping.")

# Function to select random images for training
def select_random_train_images(image_dir, num_samples=400):
    all_images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    num_images = len(all_images)

    if num_images < num_samples:
        print(f"Warning: Only {num_images} images available for training. Adjusting to select all.")
        num_samples = num_images  # Adjust to available number of images

    return random.sample(all_images, num_samples)

# Select random train images
random_train_images = select_random_train_images(train_image_dir, num_samples=400)

# Prepare YOLO data by copying images and labels to the target folder
prepare_yolo_data(train_image_dir, train_label_dir, train_target_image_dir, train_target_label_dir, random_train_images)


**Train the YOLO Model**

In [ ]:
!pip install -U torch torchvision
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 17022 (delta 122), reused 133 (delta 74), pack-reused 16805 (from 1)
Receiving objects: 100% (17022/17022), 15.74 MiB | 18.94 MiB/s, done.
Resolving deltas: 100% (11651/11651), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.1/877.1 kB 24.4 MB/s eta 0:00:00


In [ ]:
yaml_content = """
train: /content/drive/MyDrive/Portfolio 6/yolo_data/train
val: /content/drive/MyDrive/Portfolio 6/images/test

nc: 1
names: ['graffiti']
"""

with open('/content/drive/MyDrive/Portfolio 6/yolo_data/yolov5_data.yaml', 'w') as file:
    file.write(yaml_content)



In [ ]:
%cd /content/yolov5

/content/yolov5


**Change Grafitti to 0**

In [4]:
import os

# Define the directory where your label files are located
label_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/train/labels'  # Update this path as needed
class_id = 0  # Update this to the appropriate class ID for "Graffiti"

# Iterate through each label file in the specified directory
for filename in os.listdir(label_dir):
    if filename.endswith('.txt'):
        # Full path to the label file
        file_path = os.path.join(label_dir, filename)

        # Read the content of the label file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Open the file again to write updated content
        with open(file_path, 'w') as file:
            for line in lines:
                # Replace "Graffiti" with the class ID if it appears
                if 'Graffiti' in line:
                    # Split the line into parts
                    parts = line.strip().split()
                    # Update the class ID to the numeric class ID
                    parts[0] = str(class_id)  # Update class ID
                    # Write the modified line back to the file
                    file.write(' '.join(parts) + '\n')
                else:
                    # Write the line back unchanged if it doesn't contain "Graffiti"
                    file.write(line)

print("Replacement complete.")

Replacement complete.


In [ ]:
import os

# Define the directory where your label files are located
label_dir = '/content/drive/MyDrive/Portfolio 6/labels/test'  # Update this path as needed
class_id = 0  # Update this to the appropriate class ID for "Graffiti"

# Iterate through each label file in the specified directory
for filename in os.listdir(label_dir):
    if filename.endswith('.txt'):
        # Full path to the label file
        file_path = os.path.join(label_dir, filename)

        # Read the content of the label file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Open the file again to write updated content
        with open(file_path, 'w') as file:
            for line in lines:
                # Replace "Graffiti" with the class ID if it appears
                if 'Graffiti' in line:
                    # Split the line into parts
                    parts = line.strip().split()
                    # Update the class ID to the numeric class ID
                    parts[0] = str(class_id)  # Update class ID
                    # Write the modified line back to the file
                    file.write(' '.join(parts) + '\n')
                else:
                    # Write the line back unchanged if it doesn't contain "Graffiti"
                    file.write(line)

print("Replacement complete.")

Replacement complete.


In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data "/content/drive/MyDrive/Portfolio 6/yolo_data/yolov5_data.yaml" --weights yolov5s.pt --cache --save-period 1


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-10-24 06:51:48.008871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 06:51:48.041176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 06:51:48.050916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi

**Select random 40 test images and compute IoU**

In [ ]:
from PIL import Image
import pandas as pd
import torch
import os
from pathlib import Path
import random

def calculate_iou(box1, box2):
    # box format: [x1, y1, x2, y2]
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area
    return iou

# Evaluate on 40 test images and save results in CSV
def evaluate_model(model_path, test_images_dir, output_csv="/content/drive/MyDrive/Portfolio 6/iou_results.csv"):

    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)  # Load trained YOLO model

    # Get test images
    test_images = list(Path(test_images_dir).glob("*.jpg"))

    results = []
    for image_path in random.sample(test_images, 40):  # Evaluate 40 random test images
        img = Image.open(image_path)
        result = model(img)
        predictions = result.pred[0]  # Get predictions

        # Get the corresponding label file
        label_file = image_path.with_suffix('.txt')

        # Extract ground truth bounding boxes from label file
        ground_truth_boxes = []
        if label_file.exists():
            with open(label_file, 'r') as f:
                for line in f:
                    # Assuming label format: class x_center y_center width height
                    # Convert to [x_min, y_min, x_max, y_max]
                    label_data = list(map(float, line.strip().split()))
                    x_center, y_center, width, height = label_data[1:]
                    img_width, img_height = img.size  # Get image dimensions

                    x_min = int((x_center - width / 2) * img_width)
                    y_min = int((y_center - height / 2) * img_height)
                    x_max = int((x_center + width / 2) * img_width)
                    y_max = int((y_center + height / 2) * img_height)

                    ground_truth_boxes.append([x_min, y_min, x_max, y_max])


        if len(predictions) > 0:
            for pred in predictions:
                confidence = pred[4].item()

                # Calculate IoU with each ground truth box
                max_iou = 0
                for gt_box in ground_truth_boxes:
                    iou = calculate_iou(gt_box, [pred[0].item(), pred[1].item(), pred[2].item(), pred[3].item()])
                    max_iou = max(max_iou, iou) # Get the highest IoU for this prediction

                results.append([image_path.name, confidence, max_iou])
        else:
            # No graffiti detected, IoU is 0
            results.append([image_path.name, 0, 0])
    # Save results to CSV
    df = pd.DataFrame(results, columns=["image_name", "confidence", "iou"])
    df.to_csv(output_csv, index=False)

# Call the evaluation function
evaluate_model('runs/train/exp/weights/best.pt', test_image_dir)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-24 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common

In [7]:



def iterative_training(num_iterations):
    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}")

        # Select random images for training and testing
        random_train_images = select_random_train_images(train_image_dir, num_samples=400)
        prepare_yolo_data(train_image_dir, train_label_dir, train_target_image_dir, train_target_label_dir, random_train_images)

        # Train the model
        !python train.py --img 640 --batch 16 --epochs 50 --data "/content/drive/MyDrive/Portfolio 6/yolo_data/yolov5_data.yaml" --weights runs/train/exp/weights/best.pt --cache --save-period 1

        # Evaluate the model
        evaluate_model('runs/train/exp/weights/best.pt', test_image_dir, output_csv='/content/drive/MyDrive/Portfolio 6/iou_results_train.csv')

        # Check if IoU for 80% of images > 90%
        iou_df = pd.read_csv(output_csv)
        high_iou_count = sum(iou_df['iou'] > 0.9)
        if high_iou_count / len(iou_df) > 0.8:
            print("Stopping early as IoU threshold met")
            break

# Run the iterative training process
iterative_training(10)



Iteration 1


NameError: name 'train_image_dir' is not defined